In [24]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,    
    )

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template ="""
    You are a machine that summarizes movies in three emojis.

    {chat_history}
    human:{Movie}
    you:
"""
examples = [
    {
        "Movie": "Top Gun",
        "answer": "✈️🏍️🔥",
    },
    {
        "Movie": "Frozen",
        "answer": "❄️👭👑",
    },
    {
        "Movie": "Dune",
        "answer": "🏜️🪐👑",
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Summarize this {Movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a machine that summarizes movies in three emojis."),
        example_prompt,
        ("human", "Summarize this {Movie}?"),
    ]
)
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | llm

def invoke_chain(Movie):
    result = chain.invoke({"Movie":Movie})
    memory.save_context(
        {"input":Movie},
        {"output":result.content},
    )
    print(result)


In [25]:
invoke_chain("Mission Impossible")

content='🕵️\u200d♂️💣🔥'


In [26]:
invoke_chain("007")

content='🕵️\u200d♂️🔫🍸'


In [27]:
invoke_chain("Iron man")

content='🔥💥🦾'
